In [1]:
!pip -q install -U langchain_community

In [2]:
!pip -q install -U langchain langgraph 

In [1]:
import os

os.environ["GROQ_API_KEY"]

In [2]:
from langchain_groq import ChatGroq

groq_model = ChatGroq(model="llama3-70b-8192")

ModuleNotFoundError: No module named 'langchain_groq'

In [2]:
from langchain_community.llms import Ollama

ollama_model = Ollama(model='llama3')

In [5]:
model = groq_model

In [ ]:
model = ollama_model

In [6]:
model.invoke('Tell me a joke')

"Here's one:\n\nWhy couldn't the bicycle stand up by itself?\n\n(Wait for it...)\n\nBecause it was two-tired!\n\nHope that made you smile!"

In [3]:
groq_model.invoke('Tell me a joke')

AIMessage(content="Why couldn't the bicycle stand up by itself?\n\nBecause it was two-tired!", response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 14, 'total_tokens': 31, 'completion_time': 0.055098735, 'prompt_time': 0.006721211, 'queue_time': None, 'total_time': 0.061819946}, 'model_name': 'llama3-70b-8192', 'system_fingerprint': 'fp_753a4aecf6', 'finish_reason': 'stop', 'logprobs': None}, id='run-006c7923-731b-4841-bbe9-5fd94df78a24-0')

In [6]:
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser 
from langchain_core.runnables import (
    RunnableLambda,
    RunnableParallel,
    RunnablePassthrough,
)

from typing import Callable 

In [28]:
# user_request_categories = {
#     'project_help': 'when someone is asking for help with project from list: starknet, wormhole',
#     'investment_help': 'when someone is asking for help with investments in crypto',
#     'off_topic': 'it doesnt relate to any other category'
# }

user_request_categories = {
    'execute_action': 'when user wants to perform an action or asking how he can perform an action',
    'ask_for_data': 'when user is asking for a data',
    'off_topic': 'it doesnt relate to any other category'
}

In [29]:
categorize_request_prompt = PromptTemplate(
    template= """<|begin_of_text|><|start_header_id|>system<|end_header_id|>
    You are User Request Categorizer Agent You are master at understanding what a user wants to ask in chat and are able to categorize it in a useful way

    <|eot_id|><|start_header_id|>user<|end_header_id|>
    Conduct a comprehensive analysis of the user message provided and categorize into one of the following categories:
        {catrgories_desctiptions}
        
            Output ONLY a single cetgory only from the types: {category_list} \
            eg:
            off_topic \

    USER MESSAGE:\n\n {user_message} \n\n
    <|eot_id|>
    <|start_header_id|>assistant<|end_header_id|>
    """,
    input_variables=["user_message", "catrgories_desctiptions"]
)

catrgories_desctiptions = '\n'.join(map(lambda x: f"{x[0]} - {x[1]}", user_request_categories.items()))
category_list = ', '.join(map(lambda x: f'"{x}"', user_request_categories.keys()))

# chain = { "user_message": RunnablePassthrough() }
# chain.invoke('blabla', config={"catrgories_desctiptions": catrgories_desctiptions, "category_list": category_list})

user_message_category_generator = (
    {
        "catrgories_desctiptions": lambda _: catrgories_desctiptions,
        "category_list": lambda _: category_list,
        "user_message": RunnablePassthrough()
    }
    | categorize_request_prompt
    | model
    | StrOutputParser()
)


In [8]:
prompts = [
    'what is Sui and how it is better than others??',
    'Is there any NFT projects on Sui?',
    'can I buy a test NFTs on SUi?'
]

In [30]:
for prompt in prompts:
    print(f"{prompt} - {user_message_category_generator.invoke(prompt)}")

what is Sui and how it is better than others?? - ask_for_data
Is there any NFT projects on Sui? - ask_for_data
can I buy a test NFTs on SUi? - execute_action


In [7]:
USER_MESSAGE = 'Hi! How can I earn yield on my ETH?'

user_message_category_generator.invoke({"user_message": USER_MESSAGE})

'investment_help'

In [58]:
test_investment_help = [
    'Hi! How can I earn yield on my ETH?',
    'I need a conservative strategy for my long-term ETH holdings.',
    'Hey, how can i earn with my eth?',
    'what are investment strategies for eth?',
    'where to invest eth?',
    'what investment opportunities are for eth?',
    'any ideas for income with eth?',
    'how to make money on my crypto?',
    'how to get rich in crypto?',
    'suggest me with investment?',
    'suggest me where to invest in crypto?'
]

test_project_help = [
    # wormhole
    'What can Wormhole be used for?',
    'What is the role of VAAs in the Wormhole protocol, and how are they validated and processed?',
    'What are the key on-chain components involved in the Wormhole protocol, and how do they facilitate cross-chain communication?',
    'What are the off-chain components in the Wormhole protocol, and what roles do they play in facilitating cross-chain communication?',
    'How do Core Contracts function within the Wormhole protocol, and what roles do the sending and receiving sides of these contracts play?',
    'Hello, can you tell me about Wormhole Guardian Network properties?',
    'Hello. maybe you know something about wormhole Connect or something like that?',
    'Can you tell me how the wormhole contract works',
    'How are VAAs uniquely identified, and how can they be retrieved in the Wormhole protocol? Please write it with details',
    'Please tell me about the blockchains supported by Wormhole',
    'can you write me properties of avalanche and Ethereum blockchains supported by wormhole',
    'Please tell me about wormhole tutorials.',
    'What the difference between wormhole token bridge and NFT bridge?',
    'what can you tell me about Standard Relayer and Specialized Relayer, which is better?',

    #starknet
    'how does starknet works?',
    'Hi, Please tell me about Starknet benefits',
    'which tolls I need to for working with Starknet?',
    'How can I install scarb on Windows? Please write me some tutorial',
    'Can you tell me something about Transaction flow in Starknet?',
    'Can you tell me about StarkGate Bridge?',
    'How can I transfer some tokens from Layer1 to layer2? Please write me instruction.',
    'Tell me about Starknet smart contract',
    'Can you tell me what the difference between Starkli and Scarb',
    'What between installation Scarb on  MacOS and Windows? Which is faster?',
    'Can you tell me about first and last release versions of starknet',
    'What the difference between L1 AND L2 contracts on Starknet',
    'What The difference between API providers and Node Providers?',
    'Can you tell me about Starknet cryptography? What the Difference between Stark Field and Stark curve',
]

test_offtop = [
    'Why is the sky blue?',
    'what is the current price of bitcoin?',
    'what is uniswap?'
]


In [6]:
user_messages = [
   
]

for user_message in user_messages:
    print(f"{user_message} - {user_message_category_generator.invoke({'user_message': user_message})}")

Hi! How can I earn yield on my ETH? - investment_help
I need a conservative strategy for my long-term ETH holdings. - investment_help
Hey, how can i earn with my eth? - investment_help
what are investment strategies for eth? - investment_help
where to invest eth? - investment_help
what investment opportunities are for eth? - investment_help
any ideas for income with eth? - 'investment_help'
how to make money on my crypto? - investment_help
how to get rich in crypto? - investment_help
suggest me with investment? - investment_help
suggest me where to invest in crypto? - investment_help
What can Wormhole be used for? - 'project_help'
What is the role of VAAs in the Wormhole protocol, and how are they validated and processed? - 'project_help'
What are the key on-chain components involved in the Wormhole protocol, and how do they facilitate cross-chain communication? - 'project_help'
What are the off-chain components in the Wormhole protocol, and what roles do they play in facilitating cros

In [12]:
user_messages = [
    'Hi! How can I earn yield on my ETH?',
    'I need a conservative strategy for my long-term ETH holdings.',
    'Hey, how can i earn with my eth?',
    'what are investment strategies for eth?',
    'where to invest eth?',
    'what investment opportunities are for eth?',
    'any ideas for income with eth?',
    'how to make money on my crypto?',
    'how to get rich in crypto?',
    'suggest me with investment?',
    'suggest me where to invest in crypto?',

    # wormhole
    'What can Wormhole be used for?',
    'What is the role of VAAs in the Wormhole protocol, and how are they validated and processed?',
    'What are the key on-chain components involved in the Wormhole protocol, and how do they facilitate cross-chain communication?',
    'What are the off-chain components in the Wormhole protocol, and what roles do they play in facilitating cross-chain communication?',
    'How do Core Contracts function within the Wormhole protocol, and what roles do the sending and receiving sides of these contracts play?',
    'Hello, can you tell me about Wormhole Guardian Network properties?',
    'Hello. maybe you know something about wormhole Connect or something like that?',
    'Can you tell me how the wormhole contract works',
    'How are VAAs uniquely identified, and how can they be retrieved in the Wormhole protocol? Please write it with details',
    'Please tell me about the blockchains supported by Wormhole',
    'can you write me properties of avalanche and Ethereum blockchains supported by wormhole',
    'Please tell me about wormhole tutorials.',
    'What the difference between wormhole token bridge and NFT bridge?',
    'what can you tell me about Standard Relayer and Specialized Relayer, which is better?',

    # starknet
    'how does starknet works?',

    # offtop
    'Why is the sky blue?',
    'what is the current price of bitcoin?'
]

for user_message in user_messages:
    print(f"{user_message} - {user_message_category_generator.invoke({'user_message': user_message})}")

Hi! How can I earn yield on my ETH? - investment_help
I need a conservative strategy for my long-term ETH holdings. - investment_help
Hey, how can i earn with my eth? - investment_help
what are investment strategies for eth? - investment_help
where to invest eth? - investment_help
what investment opportunities are for eth? - 'investment_help'
any ideas for income with eth? - investment_help
how to make money on my crypto? - investment_help
how to get rich in crypto? - investment_help
suggest me with investment? - 'investment_help'
suggest me where to invest in crypto? - investment_help
What can Wormhole be used for? - 'project_help'
What is the role of VAAs in the Wormhole protocol, and how are they validated and processed? - 'project_help'
What are the key on-chain components involved in the Wormhole protocol, and how do they facilitate cross-chain communication? - 'project_help'
What are the off-chain components in the Wormhole protocol, and what roles do they play in facilitating cr

In [1]:
from langchain.schema import Document
from langgraph.graph import END, StateGraph

In [23]:
from typing_extensions import TypedDict
from typing_extensions import Literal
# from typing import List

class ChatMessageFlowState(TypedDict):
    user_message: str
    message_category: str
    num_steps : int

In [49]:
def categorize_user_request(state: ChatMessageFlowState):
    # print('Categorizing message')

    user_message = state['user_message']
    num_steps = int(state['num_steps'])
    num_steps += 1

    message_category = user_message_category_generator.invoke({'user_message': user_message})
    # print(message_category)

    return {'message_category': message_category, 'num_steps': num_steps}

In [50]:
def categorize_user_router(state: ChatMessageFlowState):
    return state['message_category']

In [51]:
def dummy_1(state: ChatMessageFlowState):
    # print('dummy 1')
    # print(state['message_category'])

    num_steps = int(state['num_steps'])
    num_steps += 1

    return {'num_steps': num_steps}

In [52]:
def dummy_2(state: ChatMessageFlowState):
    # print('dummy 2')
    # print(state['message_category'])

    num_steps = int(state['num_steps'])
    num_steps += 1

    return {'num_steps': num_steps}

In [53]:
def dummy_3(state: ChatMessageFlowState):
    # print('dummy 3', state['message_category'])

    num_steps = int(state['num_steps'])
    num_steps += 1

    return {'num_steps': num_steps}

In [54]:
workflow = StateGraph(ChatMessageFlowState)

workflow.add_node('categorize_user_request', categorize_user_request)
workflow.add_node('dummy_1', dummy_1)
workflow.add_node('dummy_2', dummy_2)
workflow.add_node('dummy_3', dummy_3)

workflow.add_conditional_edges(
    'categorize_user_request',
    categorize_user_router,
        {
            'project_help': 'dummy_1',
            'investment_help': 'dummy_2',
            'off_topic': 'dummy_3'
        }
    )

workflow.set_entry_point('categorize_user_request')

# workflow.add_node('categorize_user_request', categorize_user_request)

# workflow.add_edge(['dummy_1', 'dummy_2', 'dummy_3'], END)
workflow.add_edge('dummy_1', END)
workflow.add_edge('dummy_2', END)
workflow.add_edge('dummy_3', END)

In [55]:
app = workflow.compile()

In [39]:
inputs = {'user_message': 'Hi! How can I earn yield on my ETH?', "num_steps": 0}
result = app.invoke(inputs)

print(result['message_category'])

Categorizing message
investment_help
dummy 2
investment_help
investment_help


In [69]:
investment_help_count = 0
investment_help_failed = []
print('checking invesment help')
for user_message in test_investment_help:
    inputs = {'user_message': user_message, "num_steps": 0}
    # print(user_message)
    result = app.invoke(inputs)
    investment_help_count += 1
    if result['message_category'] != 'investment_help':
        investment_help_failed.append({user_message: result['message_category']})

project_help_count = 0
project_help_failed = []
print('checking project help')
for user_message in test_project_help:
    inputs = {'user_message': user_message, "num_steps": 0}
    # print(user_message)
    result = app.invoke(inputs)
    project_help_count += 1
    if result['message_category'] != 'project_help':
        project_help_failed.append({user_message: result['message_category']})


offtop_count = 0
offtop_failed = []
print('checking offtop')
for user_message in test_offtop:
    inputs = {'user_message': user_message, "num_steps": 0}
    # print(user_message)
    result = app.invoke(inputs)
    offtop_count += 1
    if result['message_category'] != 'off_topic':
        offtop_failed.append({user_message: result['message_category']})

print('investment help:', f"{investment_help_count - len(investment_help_failed)} of {investment_help_count}")
print(f'failed: {investment_help_failed}')


print('project help:', f"{project_help_count - len(project_help_failed)} of {project_help_count}")
print(f'failed: {project_help_failed}')

print('offtop:', f"{offtop_count - len(offtop_failed)} of {offtop_count}")
print(f'failed: {offtop_failed}')


checking invesment help
checking project help
checking offtop
investment help: 11 of 11
failed: []
project help: 28 of 28
failed: []
offtop: 1 of 3
failed: [{'what is the current price of bitcoin?': 'investment_help'}, {'what is uniswap?': 'investment_help'}]
